In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sismos = pd.read_csv(".//SSNMX_catalogo_20170901_20210510_lat12d85_18d66_lon-97d26_-88d69.csv",  skiprows=4)
sismos= sismos.dropna()

In [3]:
sismos["timestamp"] = sismos['Fecha'] + ' ' +sismos['Hora']
sismos["timestamp"] = sismos["timestamp"].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
sismos.drop(['Fecha UTC','Hora UTC', 'Estatus'], 1)
sismos["date_seconds"] = sismos["timestamp"].apply(lambda d: (d - sismos["timestamp"][0]).total_seconds())
sismos.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus,timestamp,date_seconds
0,2017-09-01,01:26:16,3.4,15.9832,-96.9505,25.8,"18 km al NORESTE de PUERTO ESCONDIDO, OAX",2017-09-01,06:26:16,revisado,2017-09-01 01:26:16,0.0
1,2017-09-01,01:29:47,3.5,16.6938,-95.1257,94.8,"15 km al NORTE de CD IXTEPEC, OAX",2017-09-01,06:29:47,revisado,2017-09-01 01:29:47,211.0
2,2017-09-01,02:10:55,3.8,15.5425,-92.4787,187.6,"32 km al NOROESTE de MOTOZINTLA, CHIS",2017-09-01,07:10:55,revisado,2017-09-01 02:10:55,2679.0
3,2017-09-01,02:15:28,3.5,16.0508,-97.1588,26.3,"23 km al NOROESTE de PUERTO ESCONDIDO, OAX",2017-09-01,07:15:28,revisado,2017-09-01 02:15:28,2952.0
4,2017-09-01,02:24:18,3.6,17.1730,-94.9790,121.0,"34 km al NORTE de MATIAS ROMERO, OAX",2017-09-01,07:24:18,revisado,2017-09-01 02:24:18,3482.0


In [4]:
def mcvalue(m, mbin):
    density = np.histogram(m, range = (min(m) - 1, max(m) + 1), density=True)
    return density[1][np.argmax(density[0])]

In [5]:
m = sismos['Magnitud'].values
m = m[~np.isnan(m)]
mbin = 0.1 
Mc = mcvalue(m, mbin)
# Aki ML estimates
B = np.log10(np.e)/(m[m>Mc-mbin/2].mean() - (Mc-mbin/2))
#d = len(m[m>0])
d = 2*B
c = 1

#Arbitrary c. (We don't know what it is)
#We're also not entirely sure about the d. We need to find more sources.
#dist_matrix=dist_quakes_sparse(sismos, B, 1, d)
#plt.imshow(dist_matrix[-1000:,:])

In [24]:
def dist_quakes(sismo_i, sismo_j, **kwargs):
    
    c = 1
    B = 0.6160988624731996
    d = 2*B
    #tau = sismo_j[10] - sismo_i[10]
    #tau = tau.total_seconds() # The time scale will be in seconds
    tau = sismo_j[4] - sismo_i[4]
    tau = np.abs(tau)
    lat1 = sismo_i[1]
    lat2 = sismo_j[1]
    lon1 =  sismo_i[2]
    lon2 = sismo_j[2]
    
    rad = np.pi/180
    dlat= lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(np.sin(rad*dlat/2))**2 + np.cos(rad*lat1)*np.cos(rad*lat2)*(np.sin(rad*dlon/2))**2
    distancia=2*R*np.arcsin(np.sqrt(a))
    
    r = np.sqrt(distancia**2 + (sismo_j[3]-sismo_i[3])**2 ) 
    delta_m = sismo_j[0] - sismo_i[0] 
    return kwargs["metric_params"]["c"]*tau*(r**kwargs["metric_params"]["d"]) *(10**(-kwargs["metric_params"]["B"] * delta_m))
    #return c*tau*(r**d) *(10**(-B*delta_m))



In [25]:
test = sismos.drop(sismos.columns.difference(['Magnitud', 'Latitud', 'Longitud','Profundidad','date_seconds']), 
                   1, inplace=False)
test = test.to_numpy()

array([  3.4   ,  15.9832, -96.9505,  25.8   ,   0.    ])

In [17]:
test[0][11]

0.0

In [26]:
dist_quakes(test[6], test[16])

6979382.88476807

## KNN NEAREST 

In [8]:
import sklearn
from sklearn.neighbors import NearestNeighbors


In [27]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree', metric = dist_quakes, 
                        metric_params={"c": c, "d": d, "B": B})

In [28]:
nbrs.fit(test)

NearestNeighbors(algorithm='ball_tree',
                 metric=<function dist_quakes at 0x0000022F4D414EE8>,
                 n_neighbors=10)

In [ ]:
nbrs.kneighbors(test)